In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
for i in train.columns:
    tt = Counter(train[i].isnull().values)
    print(tt, i)
print('*'*30)
for i in test.columns:
    tt = Counter(test[i].isnull().values)
    print(tt, i)

Counter({False: 891}) PassengerId
Counter({False: 891}) Survived
Counter({False: 891}) Pclass
Counter({False: 891}) Name
Counter({False: 891}) Sex
Counter({False: 714, True: 177}) Age
Counter({False: 891}) SibSp
Counter({False: 891}) Parch
Counter({False: 891}) Ticket
Counter({False: 891}) Fare
Counter({True: 687, False: 204}) Cabin
Counter({False: 889, True: 2}) Embarked
******************************
Counter({False: 418}) PassengerId
Counter({False: 418}) Pclass
Counter({False: 418}) Name
Counter({False: 418}) Sex
Counter({False: 332, True: 86}) Age
Counter({False: 418}) SibSp
Counter({False: 418}) Parch
Counter({False: 418}) Ticket
Counter({False: 417, True: 1}) Fare
Counter({True: 327, False: 91}) Cabin
Counter({False: 418}) Embarked


In [5]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [6]:
train['Embarked'].replace(np.NaN, 'S', inplace=True)

In [7]:
train['FamilySize']=train['SibSp']+train['Parch']
test['FamilySize']=test['SibSp']+test['Parch']
for_submission = test['PassengerId']

train=train.drop(['Cabin','PassengerId','Name', 'Ticket', 'SibSp', 'Parch'],axis=1)
test=test.drop(['Cabin','PassengerId','Name', 'Ticket', 'SibSp', 'Parch'],axis=1)

In [8]:
for i in train.columns:
    tt = Counter(train[i].isnull().values)
    print(tt, i)
print('*'*30)
for i in test.columns:
    tt = Counter(test[i].isnull().values)
    print(tt, i)

Counter({False: 891}) Survived
Counter({False: 891}) Pclass
Counter({False: 891}) Sex
Counter({False: 714, True: 177}) Age
Counter({False: 891}) Fare
Counter({False: 891}) Embarked
Counter({False: 891}) FamilySize
******************************
Counter({False: 418}) Pclass
Counter({False: 418}) Sex
Counter({False: 332, True: 86}) Age
Counter({False: 417, True: 1}) Fare
Counter({False: 418}) Embarked
Counter({False: 418}) FamilySize


In [9]:
train['Age'].fillna(train['Age'].mean(), inplace=True)
test['Age'].fillna(test['Age'].mean(), inplace=True)
test['Fare'].fillna(test['Fare'].mean(), inplace=True)

In [10]:
my_label = LabelEncoder()
train['Sex'] = my_label.fit_transform(train['Sex'])
test['Sex'] = my_label.fit_transform(test['Sex'])
my_label = LabelEncoder()
train['Embarked'] = my_label.fit_transform(train['Embarked'])
test['Embarked'] = my_label.fit_transform(test['Embarked'])
train

,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize
0,0,3,1,22.000000,7.2500,2,1
1,1,1,0,38.000000,71.2833,0,1
2,1,3,0,26.000000,7.9250,2,0
3,1,1,0,35.000000,53.1000,2,1
4,0,3,1,35.000000,8.0500,2,0
...,...,...,...,...,...,...,...
886,0,2,1,27.000000,13.0000,2,0
887,1,1,0,19.000000,30.0000,2,0
888,0,3,0,29.699118,23.4500,2,3
889,1,1,1,26.000000,30.0000,0,0


In [11]:
bucket=20
train['Age'] = [int(i) for i in ((train['Age'].values)//bucket)]
test['Age'] = [int(i) for i in ((test['Age'].values)//bucket)]
train['Fare'] = [int(i) for i in ((train['Fare'].values)//bucket)]
test['Fare'] = [int(i) for i in ((test['Fare'].values)//bucket)]

print(train.head())
print(test.head())

   Survived  Pclass  Sex  Age  Fare  Embarked  FamilySize
0         0       3    1    1     0         2           1
1         1       1    0    1     3         0           1
2         1       3    0    1     0         2           0
3         1       1    0    1     2         2           1
4         0       3    1    1     0         2           0
   Pclass  Sex  Age  Fare  Embarked  FamilySize
0       3    1    1     0         1           0
1       3    0    2     0         2           1
2       2    1    3     0         1           0
3       3    1    1     0         2           0
4       3    0    1     0         2           2


In [12]:
X = train.drop(['Survived'], axis=1)
y = train['Survived']

In [13]:
train_X, val_X, train_y, val_y = train_test_split(X, y)

In [14]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(train_X, train_y)
prediction = rf.predict(val_X)
print(mean_absolute_error(val_y, prediction), accuracy_score(val_y, prediction))

0.16143497757847533 0.8385650224215246


In [15]:
from xgboost import XGBClassifier

XGB = XGBClassifier()
XGB.fit(train_X, train_y)
prediction = XGB.predict(val_X)
print(mean_absolute_error(val_y, prediction), accuracy_score(val_y, prediction))

0.16591928251121077 0.8340807174887892


In [16]:
final_rf = RandomForestClassifier(random_state=1)
final_xgb = XGBClassifier()

final_rf.fit(X,y)
final_xgb.fit(X,y)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [17]:
vals = final_rf.predict(test)
file = pd.DataFrame({'PassengerId':for_submission, 'Survived':vals})
file.to_csv('submission_rf.csv', index = False)
file.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [18]:
vals = final_xgb.predict(test)
file = pd.DataFrame({'PassengerId':for_submission, 'Survived':vals})
file.to_csv('submission_xgb.csv', index = False)
file.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
